# Deprecated stuff

In [ ]:
library(tidyverse)

# Installed just by typing install.packages('bnlearn')
library(bnlearn)

# This was a bit more complicated to install:
# install.packages("BiocManager")
# BiocManager::install("Rgraphviz")
library(Rgraphviz)

The following two function compute the probability P(B,D), however they are specific for two particular Bayesian Network. They were made just as a basis for the general function present in project.ipynb

In [ ]:
ex <- read.table("./dataset/cooper.txt", header = TRUE, stringsAsFactors = TRUE)

# This function computes the humongous formula for P(B,D)
# This function is not general, I'm assuming that the variable Xj depends only on Xj-1.
# Of course it works for this particular case
ex.probmodel1 <- function(D)
    {
        nvar <- length(names(D))
        
        # For n = 1
        col1 <- dplyr::pull(D, 1)
        nunique1 <- length(unique(col1))
        prod <- factorial(nunique1-1)
        den <- nunique1 - 1
        
        for(i in 1:nunique1)
            {
                prod <- prod*factorial(length(col1[col1 == unique(col1)[i]]))
                den <- den + length(col1[col1 == unique(col1)[i]])
            }
        prod <- prod/factorial(den)
    
        # For n >= 1
        for(i in 2:nvar)
            {
                col <- dplyr::pull(D, i)
                prevcol <- dplyr::pull(D, i-1)
                
                r <- length(unique(col))
                q <- length(unique(prevcol))
                for(j in 1:q)
                    {
                        nij <- 0 
                        for(k in 1:r)
                            {
                                nijk <- sum(prevcol == unique(prevcol)[j] & col == unique(col)[k])
                                nij <- nij + nijk
                                prod <- prod*factorial(nijk)
                            }
                    
                        prod <- prod*factorial(r - 1)/factorial(nij + r - 1)
                    }
                
            }
    
        return(prod)
    }


# This function is very similar for the previous one but it's adapted for this other network structure
ex.probmodel2 <- function(D)
    {
        nvar <- length(names(D))
        
        # For n = 1
        col1 <- dplyr::pull(D, 1)
        nunique1 <- length(unique(col1))
        prod <- factorial(nunique1-1)
        den <- nunique1 - 1
        
        for(i in 1:nunique1)
            {
                prod <- prod*factorial(length(col1[col1 == unique(col1)[i]]))
                den <- den + length(col1[col1 == unique(col1)[i]])
            }
        prod <- prod/factorial(den)
    
        # For n >= 1
        for(i in 2:nvar)
            {
                col <- dplyr::pull(D, i)
                prevcol <- dplyr::pull(D, 1)
                
                r <- length(unique(col))
                q <- length(unique(prevcol))
                for(j in 1:q)
                    {
                        nij <- 0 
                        for(k in 1:r)
                            {
                                nijk <- sum(prevcol == unique(prevcol)[j] & col == unique(col)[k])
                                nij <- nij + nijk
                                prod <- prod*factorial(nijk)
                            }
                    
                        prod <- prod*factorial(r - 1)/factorial(nij + r - 1)
                    }
                
            }
    
        return(prod)
    }


In [ ]:
ex.bn1 <- model2network("[X1][X2|X1][X3|X2]")
ex.bn2 <- model2network("[X1][X2|X1][X3|X1]")

In [ ]:
graphviz.plot(ex.bn1,layout = 'neato')
cat('P(B1,D)=P(B1)',ex.probmodel1(ex))

In [ ]:
graphviz.plot(ex.bn2)
cat('P(B2,D)=P(B2)',ex.probmodel2(ex))